In [ ]:
import csv
import math
import pandas as pd
import random
import gzip
import torch
from sklearn import metrics
import multiprocessing
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import warnings
import torch.nn.functional as F
from torch import optim
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import make_scorer,balanced_accuracy_score,roc_curve,precision_recall_curve,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import acf
from numpy import array
from numpy import hstack
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Functions

In [ ]:
# split a multivariate sequence into samples
def Convert_format1(data, n_steps_in, n_steps_out):
    n0=np.where(data['PeriodsSepLastTwoNnZeroDemands']>0)[0][1]
    n1=n0+int((data.shape[0]-n0)*90/100)
    in_seq1=data['ZNZDemand'].values[n0:]
    in_seq2=data['LastQtte'].values[n0:]
    in_seq3=data['WeekDay'].values[n0:]
    in_seq4=data['Interval'].values[n0:]
    in_seq5=data['PeriodsSepLastTwoNnZeroDemands'].values[n0:]
    in_seq6=data['Month'].values[n0:]
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4,in_seq5,in_seq6))
    X = []
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if (out_end_ix-1) > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = list(dataset[i:end_ix,1:]), list(dataset[end_ix-1:out_end_ix-1, 0])
        X.append([seq_x,seq_y])
    size=int((n1) / 5)
    train_data=X[:n1]
    test_data=X[n1:]
    firsttrain = X[:4*size]
    firstvalid = X[4*size:n1]
    alldata=train_data+test_data
    return firsttrain,firstvalid,train_data,test_data,alldata

In [ ]:
class dataset_load(Dataset):
    def __init__(self,xy=None):
        self.x_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
# Convert Data to the defined input in the 2nd phase
def Convert_format2(data,RNN_out, n_steps_in, n_steps_out):
    n0=np.where(data['PeriodsSepLastTwoNnZeroDemands']>0)[0][1]
    n1=n0+int((data.shape[0]-n0)*90/100)
    #scaler = MinMaxScaler(feature_range = (0, 1))
    #scaler.fit(data[['Quantite']].values[n0:n1] )
    #in_seq1=scaler.transform(data[['Quantite']].values[n0:])
    in_seq1=data['Quantite'].values[n0:]
    in_seq2=data['LastQtte'].values[n0:]
    in_seq3=data['Interval'].values[n0:]
    in_seq4=RNN_out
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4))
    X = list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if (out_end_ix-1) > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix,1:].ravel(), dataset[end_ix-1:out_end_ix-1, 0].ravel()
        X.append([seq_x,seq_y])
    size=int(n1 / 5)
    train_data=X[:n1]
    test_data=X[n1:]
    firsttrain = X[:4*size]
    firstvalid = X[4*size:n1]
    return firsttrain,firstvalid,train_data,test_data

In [ ]:
# Convert Data to the defined input in the RegNetwork
def Convert_format3(data, n_steps_in, n_steps_out):
    n0=np.where(data['PeriodsSepLastTwoNnZeroDemands']>0)[0][1]
    n1=n0+int((data.shape[0]-n0)*90/100)
    scaler = MinMaxScaler(feature_range = (0, 1))
    scaler.fit(data[['Quantite']].values[n0:n1] )
    in_seq1=scaler.transform(data[['Quantite']].values[n0:])
    #in_seq1=data['Quantite'].values[n0:]
    in_seq2=data['LastQtte'].values[n0:]
    in_seq3=data['WeekDay'].values[n0:]
    in_seq4=data['Interval'].values[n0:]
    in_seq5=data['PeriodsSepLastTwoNnZeroDemands'].values[n0:]
    in_seq6=data['Month'].values[n0:]
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4,in_seq5,in_seq6))
    X = []
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if (out_end_ix-1) > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix,1:], dataset[end_ix-1:out_end_ix-1, 0]
        X.append([seq_x,seq_y])
    size=int(n1 / 5)
    train_data=X[:n1]
    test_data=X[n1:]
    firsttrain = X[:4*size]
    firstvalid = X[4*size:n1]
    return firsttrain,firstvalid,train_data,test_data,scaler

In [ ]:
def logsampler(a,b):
        x=np.random.uniform(low=0,high=1)
        y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
        return y

In [ ]:
def pb_func(row):
    best=row.idxmin(axis=0, skipna=True)
    return (best)

In [ ]:
def MASE(X,Y,F):
    N1=len(X)
    N2=len(Y)
    D1=np.sum(abs(Y-F))/N2
    D2=np.sum(abs(X[1:]-X[:-1]))/N1
    return (D1/D2)

In [ ]:
def RMSSE(X,Y,F):
    h=len(Y)
    n=len(X)
    result=np.sqrt(((1/h)*np.sum((Y-F)**2))/(1/(n-1)*np.sum((X[1:]-X[:-1])**2)))
    return (result)  

# Network 1

In [ ]:
class Deepnet(nn.Module):
    def __init__ (self,RNN,RNN_hidden_size,RNN_sigma,layer_size,N_sigma,dropprob,n_features):
        super(Deepnet,self).__init__()
        self.RNN=RNN
        self.RNN_hidden_size=RNN_hidden_size
        self.RNN_sigma=RNN_sigma
        
        self.layer_size=layer_size
        self.N_sigma=N_sigma
        
        self.input_channels=n_features
        self.dropprob=dropprob
        
        if self.RNN=='LSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiLSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        elif self.RNN=='GRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiGRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        for layer_p in self.rnn._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    torch.nn.init.normal_(self.rnn.__getattr__(p),mean=0,std=RNN_sigma)
        
        #weights between LSTM or CNN or GRU layers and fully connected layer
        self.wHidden = torch.randn(self.FC_size, self.layer_size).to(device)
        self.wHiddenBias = torch.randn(self.layer_size).to(device)
        self.wHidden.requires_grad = True
        self.wHiddenBias.requires_grad = True
        
        #weights between the fully connected layer and the output node
        self.wNeu=torch.randn(self.layer_size,1).to(device)
        self.wNeuBias=torch.randn(1).to(device) 
        torch.nn.init.xavier_uniform(self.wNeu)
        torch.nn.init.xavier_uniform(self.wHidden)
        self.wNeu.requires_grad = True
        self.wNeuBias.requires_grad = True
        
        torch.nn.init.normal_(self.wNeu,mean=0,std=self.N_sigma)
        torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.N_sigma)
        torch.nn.init.normal_(self.wHidden,mean=0,std=self.N_sigma)
        torch.nn.init.normal_(self.wHiddenBias,mean=0,std=self.N_sigma)
        
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
        self.max = torch.nn.MaxPool1d(3, stride=1) #Pooling layer (pooling size =3)
    def get_weights(self):
        ll = []
        for layer_p in self.rnn._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    ll.append(self.rnn.__getattr__(p))
        return ll
    def forward(self,x):
        x=x.permute(1,0,2)
        output,_=self.rnn(x)
        if self.RNN=='BiLSTM' or self.RNN=='BiGRU':
            Normal_RNN=output[-1, :, :self.RNN_hidden_size]
            Rev_RNN=output[0, :, self.RNN_hidden_size:]
            x = torch.cat((Normal_RNN, Rev_RNN), 1)
            x=self.dropout(x)
        else:
            x = output[-1, :, :]
            x=self.dropout(x)
        x=x @ self.wHidden + self.wHiddenBias
        x=x.clamp(min=0)
        x=self.dropout(x)
        x=x @ self.wNeu + self.wNeuBias
        return (torch.sigmoid(x))

In [ ]:
def Calibration(RNN,w1,w0,firsttrain_loader,firstvalid_loader,n_features,metric):
    print('start')
    best_AUC = 0
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    learning_steps_list = [50, 100, 150, 200, 250, 300, 350, 400]
    for number in range(40):
        # hyper-parameters
        RNN_hidden_size_list = [20, 50, 80, 100]
        RNN_hidden_size = random.choice(RNN_hidden_size_list)
        dropoutList = [0, 0.15, 0.3, 0.45]
        dropprob = random.choice(dropoutList)
        layer_size_list = [32, 64]
        layer_size = random.choice(layer_size_list)
        learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
        learning_rate=random.choice(learning_rate_list)
        RNN_sigma = logsampler(10 ** -4, 10 ** -2)
        N_sigma = logsampler(10 ** -4, 10 ** -2)
        model_auc = []
        model = Deepnet(RNN,RNN_hidden_size,RNN_sigma,layer_size,N_sigma,dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(
                model.get_weights() + [model.wNeu, model.wNeuBias, model.wHidden, model.wHiddenBias],
                lr=learning_rate)

        learning_steps = 0
        while learning_steps <= 400:
            auc = []
            model.train()
            for i, (data, target) in enumerate(firsttrain_loader):

                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)
                loss = F.binary_cross_entropy(output, target,weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                if (learning_steps % 50 == 0):

                    with torch.no_grad():
                        model.eval()
                        auc = []
                        for j, (data1, target1) in enumerate(firstvalid_loader):
                            data1 = data1.to(device)
                            target1 = target1.to(device)

                            # Forward pass
                            output = model(data1)

                            pred = output.cpu().detach().numpy().reshape(output.shape[0])
                            labels = target1.cpu().numpy().reshape(output.shape[0])
                            if output.shape[0] > 60:
                                if (metric=='ROC'):
                                    auc.append(metrics.roc_auc_score(labels, pred))
                                elif (metric=='PRC'):
                                    precision, recall, _ = precision_recall_curve(labels, pred)
                                    auc.append(metrics.auc(recall, precision))
                                else :
                                    print('Choose proper metric')
                                    break;
                        model_auc.append(np.mean(auc))

                        model.train()
            learning_steps += 1

        for n in range(8):
            AUC = model_auc[n]
            # print(AUC)
            if AUC > best_AUC:
                best_AUC = AUC
                best_learning_steps = learning_steps_list[n]
                best_LearningRate = learning_rate
                best_RNN_hidden_size=RNN_hidden_size
                best_dropprob = dropprob
                best_layer_size= layer_size
                best_RNN_sigma = RNN_sigma
                best_N_sigma=N_sigma

    print('best_AUC=', best_AUC)
    print('best_learning_steps=', best_learning_steps)
    print('best_LearningRate=', best_LearningRate)
    print('best_dropprob=', best_dropprob)
    print('best_RNN_hidden_size=', best_RNN_hidden_size)
    print('best_layer_size=', best_layer_size)
    print('best_RNN_sigma=', best_RNN_sigma)
    print('best_N_sigma=', best_N_sigma)

    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_RNN_hidden_size': best_RNN_hidden_size,
                            'best_layer_size': best_layer_size, 
                            'best_RNN_sigma': best_RNN_sigma,
                            'best_N_sigma':best_N_sigma}
    return best_hyperparameters

In [ ]:
def Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric):
    best_learning_steps=best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_N_sigma=best_hyperparameters['best_N_sigma']
    best_RNN_sigma=best_hyperparameters['best_RNN_sigma']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_AUC=0
    best_threshold=0.5
    for number_models in range(5):
        model = Deepnet(RNN,best_RNN_hidden_size,best_RNN_sigma,best_layer_size,best_N_sigma,best_dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(
                model.get_weights() + [model.wNeu, model.wNeuBias, model.wHidden, model.wHiddenBias],
                lr=best_LearningRate)

        learning_steps=0
        model.train()
        while learning_steps<=best_learning_steps:
            
            for i, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)
                loss = F.binary_cross_entropy(output, target,weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            learning_steps+=1

        with torch.no_grad():
            model.eval()
            auc=[]
            threshold=[]
            for i, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)

                pred=output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if output.shape[0]>60:
                    if (metric=='ROC'):
                        auc.append(metrics.roc_auc_score(labels, pred))
                        fpr, tpr, thresholds = roc_curve(labels, pred)
                        gmeans=np.sqrt(tpr*(1-fpr))
                    elif (metric=='PRC'):
                        precision, recall, thresholds = precision_recall_curve(labels, pred)
                        auc.append(metrics.auc(recall, precision))
                        gmeans = (2*precision*recall)/(precision+recall)
                    else :
                        print('Choose proper metric')
                        break;
                    ix = np.argmax(gmeans)
                    threshold.append(thresholds[ix])
            #             
            AUC_training=np.mean(auc)
            print('AUC on training data for model ',number_models+1,' = ',AUC_training)
            if AUC_training>best_AUC:
                best_AUC=AUC_training
                best_threshold=np.mean(threshold)
                best_model=model
    return best_model,best_threshold

In [ ]:
def test_predict(best_model,test_loader):

    with torch.no_grad():
        best_model.eval()
        auc = []

        for i, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            myprob = "\n".join(map(str, pred[:]))
            labels = target.cpu().numpy().reshape(output.shape[0])
            if output.shape[0] > 50:
                auc.append(metrics.roc_auc_score(labels, pred))

        AUC_test = np.mean(auc)
    return (labels,pred)

In [ ]:
def train_predict(best_model,alldata_loader):

    with torch.no_grad():
        best_model.eval()
        auc = []

        for i, (data, target) in enumerate(alldata_loader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            myprob = "\n".join(map(str, pred[:]))
            labels = target.cpu().numpy().reshape(output.shape[0])
    return (labels,pred)

In [ ]:
def make_confusion_matrix(cf,labels,pred,group_names=None,categories='auto',count=True,percent=True,cbar=True,xyticks=True,xyplotlabels=True,sum_stats=True,figsize=None,cmap='Blues',title=None):
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision, recall, thresholds = precision_recall_curve(labels, pred)
            Auc_PRC=metrics.auc(recall, precision)
            Auc_ROC=metrics.roc_auc_score(labels, pred)
            stats_text = "\n\nAU_PRC={:0.3f}\nAU_ROC={:0.3f}".format(Auc_PRC,Auc_ROC)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

# Network 2

In [ ]:
class Deepnet2(nn.Module):
    def __init__ (self,layer_number,layer_size,dropprob,n_features):
        super(Deepnet2,self).__init__()
                
        self.layer_size=layer_size
        self.layer_number=layer_number
        self.input_channels=n_features
        self.dropprob=dropprob
        

        
        self.FC=self.layer_size
        self.fc1=nn.Linear(self.input_channels,self.layer_size)
        if (self.layer_number>1):
            self.fc2=nn.Linear(self.layer_size,self.layer_size)
        if (self.layer_number>2):
            self.fc3=nn.Linear(self.layer_size,self.layer_size)
        self.fc_f=nn.Linear(self.layer_size,1)
        
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        if (self.layer_number>1):
            x=F.relu(self.fc2(x))
            x=self.dropout(x)
        
        if (self.layer_number>2):
            x=F.relu(self.fc3(x))
            x=self.dropout(x)
        
        x=F.relu(self.fc_f(x))

        return (x)

In [ ]:
def Calibration2(firsttrain_loader2,firstvalid_loader2,n_features2):
    print('start')
    best_MSE = 100
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    learning_steps_list = [50, 100, 150, 200, 250, 300, 350, 400]
    for number in range(40):
        # hyper-parameters
        dropoutList = [0, 0.15, 0.3, 0.45]
        dropprob = random.choice(dropoutList)
        layer_number_list=[1,2,3]
        layer_number=random.choice(layer_number_list)
        layer_size_list = [16, 32]
        layer_size = random.choice(layer_size_list)
        learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
        learning_rate=random.choice(learning_rate_list)
        model_MSE = []
        model = Deepnet2(layer_number,layer_size,dropprob,n_features2).to(device)
        optimizer = torch.optim.Adam(
                model.parameters(),
                lr=learning_rate)

        learning_steps = 0
        while learning_steps <= 400:
            model.train()
            for i, (data, target) in enumerate(firsttrain_loader2):

                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)
                loss = nn.MSELoss()
                L=loss(output,target)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
                
                if learning_steps % 50 == 0:

                    with torch.no_grad():
                        model.eval()
                        mse = []
                        for j, (data1, target1) in enumerate(firstvalid_loader2):
                            data1 = data1.to(device)
                            target1 = target1.to(device)

                            # Forward pass
                            output = model(data1)

                            pred = output.cpu().detach().numpy().reshape(output.shape[0])
                            labels = target1.cpu().numpy().reshape(output.shape[0])
                            if output.shape[0] > 60:
                                mse.append(metrics.mean_squared_error(labels, pred))
                        # print(np.mean(auc))
                        model_MSE.append(np.mean(mse))

                        model.train()
            learning_steps += 1

        for n in range(8):
            MSE = model_MSE[n]
            # print(AUC)
            if MSE < best_MSE:
                best_MSE = MSE
                best_learning_steps = learning_steps_list[n]
                best_LearningRate = learning_rate
                best_dropprob = dropprob
                best_layer_number= layer_number
                best_layer_size= layer_size

    print('best_MSE=', best_MSE)
    print('best_learning_steps=', best_learning_steps)
    print('best_LearningRate=', best_LearningRate)
    print('best_dropprob=', best_dropprob)
    print('best_layer_number=', best_layer_number)
    print('best_layer_size=', best_layer_size)

    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_layer_number': best_layer_number,
                            'best_layer_size': best_layer_size}
    return best_hyperparameters

In [ ]:
def Train_model2(best_hyperparameters,train_loader2,n_features2):
    best_learning_steps=best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_layer_number=best_hyperparameters['best_layer_number']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_MSE=100

    for number_models in range(5):
        model = Deepnet2(best_layer_number,best_layer_size,best_dropprob,n_features2).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=best_LearningRate)

        learning_steps=0
        model.train()
        while learning_steps<=best_learning_steps:
            
            for i, (data, target) in enumerate(train_loader2):
                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)
                loss = nn.MSELoss()
                L=loss(output,target)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
            learning_steps+=1

        with torch.no_grad():
            model.eval()
            mse=[]
            for i, (data, target) in enumerate(train_loader2):
                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)

                pred=output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if output.shape[0]>60:
                    mse.append(metrics.mean_squared_error(labels, pred))
                    
            MSE_Training=np.mean(mse)     

            print('MSE on training data for model ',number_models+1,' = ',MSE_Training)
            if MSE_Training<best_MSE:
                best_MSE=MSE_Training
                best_model=model
    return best_model

In [ ]:
def test_predict2(best_model,test_loader2):

    with torch.no_grad():
        best_model.eval()

        for i, (data, target) in enumerate(test_loader2):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            myprob = "\n".join(map(str, pred[:]))
            labels = target.cpu().numpy().reshape(output.shape[0])
    return (pred)

# Regression Network

In [ ]:
class RegNetwork(nn.Module):
    def __init__(self,RNN , RNN_type , RNN_size , RNN_mean , RNN_sigma , NN_number , NN_size , NN_mean , NN_sigma ,
                 dropprob , n_features):
        super(RegNetwork,self).__init__()
        self.RNN=RNN
        self.RNN_type=RNN_type
        self.RNN_size=RNN_size
        self.RNN_mean=RNN_mean        
        self.RNN_sigma=RNN_sigma
        self.NN_number=NN_number
        self.NN_mean=NN_mean
        self.NN_sigma=NN_sigma
        self.dropprob=dropprob
        self.input_channels=n_features
        self.FC_size=NN_size
        if self.RNN:
            if RNN_type=='LSTM':
                self.rnn = nn.LSTM(self.input_channels, RNN_size, num_layers=1, bidirectional=False).to(device)
                self.input_channels= RNN_size
            elif RNN_type=='BiLSTM':
                self.rnn = nn.LSTM(self.input_channels, RNN_size, num_layers=1, bidirectional=True).to(device)
                self.input_channels= 2*RNN_size
            elif RNN_type=='GRU':
                self.rnn = nn.GRU(self.input_channels, RNN_size, num_layers=1, bidirectional=False).to(device)
                self.input_channels= RNN_size
            elif RNN_type=='BiGRU':
                self.rnn = nn.GRU(self.input_channels, RNN_size, num_layers=1, bidirectional=True).to(device)
                self.input_channels= 2*RNN_size

            for layer_p in self.rnn._all_weights:
                for p in layer_p:
                    if 'weight' in p:
                        torch.nn.init.normal_(self.rnn.__getattr__(p),mean=self.RNN_mean,std=self.RNN_sigma)
        
        self.fc1=nn.Linear(self.input_channels,self.FC_size)
        torch.nn.init.normal_(self.fc1.weight,mean=self.NN_mean,std=self.NN_sigma)
        torch.nn.init.normal_(self.fc1.bias,mean=self.NN_mean,std=self.NN_sigma)
        if (self.NN_number>1):
            self.fc2=nn.Linear(self.FC_size,self.FC_size)
            torch.nn.init.normal_(self.fc2.weight,mean=self.NN_mean,std=self.NN_sigma)
            torch.nn.init.normal_(self.fc2.bias,mean=self.NN_mean,std=self.NN_sigma)
        if (self.NN_number>2):
            self.fc3=nn.Linear(self.FC_size,self.FC_size)
            torch.nn.init.normal_(self.fc3.weight,mean=self.NN_mean,std=self.NN_sigma)
            torch.nn.init.normal_(self.fc3.bias,mean=self.NN_mean,std=self.NN_sigma)
        self.fc_f=nn.Linear(self.FC_size,1)
        torch.nn.init.normal_(self.fc_f.weight,mean=self.NN_mean,std=self.NN_sigma)
        torch.nn.init.normal_(self.fc_f.bias,mean=self.NN_mean,std=self.NN_sigma)
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
        
    def forward(self,x):
        if self.RNN:
            x=x.permute(1,0,2)
            output, _ = self.rnn(x)
            if self.RNN_type=='BiLSTM' or self.RNN_type=='BiGRU':
                Normal_RNN=output[-1, :, :self.RNN_size]
                Rev_RNN=output[0, :, self.RNN_size:]
                x = torch.cat((Normal_RNN, Rev_RNN), 1)
                x=x.clamp(min=0)
                x=self.dropout(x)
            else:
                x = output[-1, :, :]
                x=self.dropout(x)
                
        
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        if (self.NN_number>1):
            x=F.relu(self.fc2(x))
            x=self.dropout(x)
        
        if (self.NN_number>2):
            x=F.relu(self.fc3(x))
            x=self.dropout(x)
        
        x=F.sigmoid(self.fc_f(x))
        #x=F.relu(self.fc_f(x))
        return (x)

In [ ]:
def Calibration3(RNN,RNN_type,firsttrain_loader3,firstvalid_loader3,n_features3):
    print('start')
    best_MSE = 100
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    learning_steps_list = [50, 100, 150, 200, 250, 300, 350, 400]
    for number in range(40):
        # hyper-parameters
        RNN_hidden_size_list = [20, 50, 80, 100]
        RNN_hidden_size = random.choice(RNN_hidden_size_list)
        dropoutList = [0, 0.15, 0.3, 0.45]
        dropprob = random.choice(dropoutList)
        RNN_mean_v = logsampler(10 ** -1, 2)
        RNN_mean=random.choice([-RNN_mean_v,RNN_mean_v])
        RNN_sigma = logsampler(10 ** -4, 1)
        weight_decay_list=[10**-5,10**-4,10**-3,10**-2]
        Weight_decay=random.choice(weight_decay_list)
        if RNN:
            layer_number=1
        else: 
            layer_number_list=[2,3]
            layer_number=random.choice(layer_number_list)
        layer_size_list = [32, 64]
        layer_size = random.choice(layer_size_list)
        learning_rate_list = [10**-6,10**-5,10**-4,10**-3]
        learning_rate=random.choice(learning_rate_list)
        NN_mean_v = logsampler(10**-1, 2)
        NN_mean=random.choice([-NN_mean_v,NN_mean_v])
        N_sigma = logsampler(10 ** -4, 1)
        model_MSE = []
        model = RegNetwork(RNN,RNN_type,RNN_hidden_size,RNN_mean,RNN_sigma,layer_number,layer_size,NN_mean,N_sigma,
                           dropprob,n_features3).to(device)

        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=Weight_decay)

        learning_steps = 0
        while learning_steps <= 400:
            mse = []
            model.train()
            for i, (data, target) in enumerate(firsttrain_loader3):

                data = data.to(device)
                target = target.to(device)

                # Forward pass
                output = model(data)
                L=F.mse_loss(target,output)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()

                if learning_steps % 50 == 0:

                    with torch.no_grad():
                        model.eval()
                        mse = []
                        for j, (data1, target1) in enumerate(firstvalid_loader3):
                            data1 = data1.to(device)
                            target1 = target1.to(device)
                            # Forward pass
                            output = model(data1)
                            
                            pred = output.cpu().detach().numpy().reshape(output.shape[0])
                            labels = target1.cpu().numpy().reshape(output.shape[0])

                            if output.shape[0] > 60:
                                mse.append(metrics.mean_squared_error(labels, pred))
                        # print(np.mean(auc))
                        model_MSE.append(np.mean(mse))

                        model.train()
            learning_steps += 1

        for n in range(8):
            MSE = model_MSE[n]
            # print(AUC)
            if MSE < best_MSE:
                best_MSE = MSE
                best_learning_steps = learning_steps_list[n]
                best_LearningRate = learning_rate
                best_dropprob = dropprob
                best_layer_number= layer_number
                best_layer_size= layer_size
                best_NN_mean=NN_mean
                best_N_sigma=N_sigma
                best_RNN_hidden_size= RNN_hidden_size
                best_RNN_mean=RNN_mean
                best_RNN_sigma=RNN_sigma
                best_weight_decay=Weight_decay

    print('best_MSE=', best_MSE)
    print('best_learning_steps=', best_learning_steps)
    print('best_LearningRate=', best_LearningRate)
    print('best_dropprob=', best_dropprob)
    print('best_weight_decay=',best_weight_decay)
    print('best_layer_number=', best_layer_number)
    print('best_layer_size=', best_layer_size)
    print('best_NN_mean=', best_NN_mean)
    print('best_N_sigma=', best_N_sigma)
    print('best_RNN_hidden_size=', best_RNN_hidden_size)
    print('best_RNN_mean=', best_RNN_mean)
    print('best_RNN_sigma=', best_RNN_sigma)
    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_weight_decay':best_weight_decay,
                            'best_layer_number': best_layer_number,
                            'best_layer_size': best_layer_size, 
                            'best_NN_mean':best_NN_mean,
                            'best_N_sigma':best_N_sigma,
                            'best_RNN_hidden_size' :best_RNN_hidden_size,
                            'best_RNN_mean':best_RNN_mean,
                            'best_RNN_sigma': best_RNN_sigma}
    return best_hyperparameters

In [ ]:
def Train_model3(RNN,RNN_type,best_hyperparameters,train_loader3,n_features3):
    best_learning_steps=best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_weight_decay=0#best_hyperparameters['best_weight_decay']
    best_layer_number=best_hyperparameters['best_layer_number']
    best_N_sigma=best_hyperparameters['best_N_sigma']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
    best_RNN_sigma=best_hyperparameters['best_RNN_sigma']
    best_RNN_mean=best_hyperparameters['best_RNN_mean']
    best_NN_mean=best_hyperparameters['best_NN_mean']
    best_MSE=100

    for number_models in range(5):
        model = RegNetwork(RNN,RNN_type,best_RNN_hidden_size,best_RNN_mean,best_RNN_sigma,best_layer_number,
                           best_layer_size,best_NN_mean,best_N_sigma,best_dropprob,n_features3).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=best_learning_steps,weight_decay=best_weight_decay)
        learning_steps=0
        model.train()
        while (learning_steps<=best_learning_steps):    
            for i, (data, target) in enumerate (train_loader3):
                data = data.to(device)
                target = target.to(device)
                # Forward pass
                output = model(data)
                L=F.mse_loss(target,output)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
            learning_steps+=1
        with torch.no_grad():
            model.eval()
            mse=[]
            for i, (data, target) in enumerate(train_loader3):

                data = data.to(device)
                target = target.to(device)
                
                # Forward pass
                output = model(data)

                pred=output.cpu().detach().numpy().reshape(output.shape[0])

                labels=target.cpu().numpy().reshape(output.shape[0])
                if (output.shape[0]>30):
                    mse.append(metrics.mean_squared_error(labels, pred))

            MSE_Training=np.mean(mse)     

            print('MSE on training data for model ',number_models+1,' = ',MSE_Training)
            if (MSE_Training<best_MSE):
                best_MSE=MSE_Training
                best_model=model
    return best_model

In [ ]:
def test_predict3(best_model,test_loader3):

    with torch.no_grad():
        best_model.eval()

        for i, (data, target) in enumerate(test_loader3):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            myprob = "\n".join(map(str, pred[:]))
            labels = target.cpu().numpy().reshape(output.shape[0])
    return (pred)

# SES

In [ ]:
def SES(ts,alpha=0.1):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    
    #level (a), periodicity(p) and forecast (f)
    f = np.full((cols),np.nan)
    
    # Initialization
    f[0] = d[0]
# Create all the t+1 forecasts
    for t in range(0,cols-1):        
        f[t+1] = alpha*d[t] + (1-alpha)*f[t]   
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Error":d-f})
    return df

In [ ]:
def evaluate_SES(DATA_Micro,n1,n2):
    Opt_SES=[0,10000]
    for i in np.arange (0,1,0.001):
        SES_train=SES(DATA_Micro.Quantite[n1:n2],alpha=i)
        if (np.sqrt(mean_squared_error(SES_train['Forecast'].values[:],DATA_Micro['Quantite'][n1:n2].values))<Opt_SES[1]):
            Opt_SES[1]=np.sqrt(mean_squared_error(SES_train['Forecast'].values[:],DATA_Micro['Quantite'][n1:n2].values))
            Opt_SES[0]=i
    return (Opt_SES[0])

# Croston

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
# Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [ ]:
def evaluate_CR(DATA_Micro,n1,n2):
    Opt_CR=[0,100]
    for i in np.arange (0,1,0.001):
        Cr_train=Croston(DATA_Micro.Quantite[n1:n2],alpha=i)
        if (np.sqrt(mean_squared_error(Cr_train['Forecast'].values[:-1],DATA_Micro['Quantite'][n1:n2].values))<Opt_CR[1]):
            Opt_CR[1]=np.sqrt(mean_squared_error(Cr_train['Forecast'].values[:-1],DATA_Micro['Quantite'][n1:n2].values))
            Opt_CR[0]=i
    return(Opt_CR[0])

# SBA

In [ ]:
def SBA(ts,extra_periods=1,alpha=0.1,beta=0.1):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
# Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*q + (1-beta)*p[t]
            f[t+1] = (1-alpha/2)*a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [ ]:
def evaluate_SBA(DATA_Micro,n1,n2):
    Opt_SBA=[0,0,100]
    for i in np.arange (0,1,0.01):
        for j in np.arange (0,1,0.01):
            SBA_train=SBA(DATA_Micro.Quantite[n1:n2],alpha=i,beta=j)
            if (np.sqrt(mean_squared_error(SBA_train['Forecast'].values[:-1],DATA_Micro['Quantite'][n1:n2].values))<Opt_SBA[2]):
                Opt_SBA[2]=np.sqrt(mean_squared_error(SBA_train['Forecast'].values[:-1],DATA_Micro['Quantite'][n1:n2].values))
                Opt_SBA[1]=j
                Opt_SBA[0]=i
    return(Opt_SBA[0],Opt_SBA[1])

# Generating results

In [ ]:
def main(path,product):
    DATA_Micro=pd.read_csv(path+product+".csv", sep=';',encoding = "ISO-8859-1")
    
    #preprocess
    #number of input lags
    autocorr=acf(DATA_Micro['Quantite'],False,100)
    max(autocorr[(autocorr<1)])
    Nlags=np.where(autocorr==max(autocorr[(autocorr<1)]))[0][0]
    
    #number of output values
    metric='PRC'
    Nout=1
    Nin=1
    n_features=5
    n_features2=3
    n_features3=5
    w1=DATA_Micro['ZNZDemand'].value_counts()[0]/DATA_Micro['ZNZDemand'].value_counts()[1]/2
    w0=1
    n1=np.where(DATA_Micro['PeriodsSepLastTwoNnZeroDemands']>0)[0][1]
    n2=2*n1+int((DATA_Micro.shape[0]-n1)*90/100)
    n_steps_in, n_steps_out = Nin, Nout
    
    RNN='BiLSTM'
    batch_size=64
    evaluate_performance=True
    #Network1
    firsttrain,firstvalid,train_data,test_data,alldata=Convert_format1(DATA_Micro,n_steps_in,n_steps_out)
    firsttrain_dataset=dataset_load(firsttrain)
    firstvalid_dataset=dataset_load(firstvalid)
    train_dataset=dataset_load(train_data)
    test_dataset=dataset_load(test_data)
    all_dataset=dataset_load(alldata)
    firsttrain_loader = DataLoader(dataset=firsttrain_dataset,batch_size=batch_size,shuffle=False)
    firstvalid_loader = DataLoader(dataset=firstvalid_dataset,batch_size=batch_size,shuffle=False)
    train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
    test_loader = DataLoader(dataset=test_dataset,batch_size=10000,shuffle=False)
    alldata_loader = DataLoader(dataset=all_dataset,batch_size=10000,shuffle=False)
    best_hyperparameters=Calibration(RNN,w1,w0,firsttrain_loader,firstvalid_loader,n_features,metric)
    best_model,best_threshold=Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric)
    labels,pred=test_predict(best_model,test_loader)
    prediction_test=np.where(pred<best_threshold,0,pred)
    prediction_test=np.where(prediction_test>best_threshold,1,prediction_test)
    labels_train,pred_train=train_predict(best_model,alldata_loader)
    prediction_train=np.where(pred_train<best_threshold,0,pred_train)
    prediction_train=np.where(prediction_train>best_threshold,1,prediction_train)
    
    #Network2
    firsttrain2,firstvalid2,train_data2,test_data2=Convert_format2(DATA_Micro,prediction_train,n_steps_in,n_steps_out)
    firsttrain_dataset2=dataset_load(firsttrain2)
    firstvalid_dataset2=dataset_load(firstvalid2)
    train_dataset2=dataset_load(train_data2)
    test_dataset2=dataset_load(test_data2)
    firsttrain_loader2 = DataLoader(dataset=firsttrain_dataset2,batch_size=batch_size,shuffle=False)
    firstvalid_loader2 = DataLoader(dataset=firstvalid_dataset2,batch_size=batch_size,shuffle=False)
    train_loader2 = DataLoader(dataset=train_dataset2,batch_size=batch_size,shuffle=False)
    test_loader2 = DataLoader(dataset=test_dataset2,batch_size=10000,shuffle=False)
    best_hyperparameters2=Calibration2(firsttrain_loader2,firstvalid_loader2,n_features2)
    best_model2=Train_model2(best_hyperparameters2,train_loader2,n_features2)
    predsHM_scaled=test_predict2(best_model2,test_loader2)
    prediction_HM=np.multiply(prediction_test,predsHM_scaled)
    
    #RegNetwork-RNN
    firsttrain3,firstvalid3,train_data3,test_data3,scaler=Convert_format3(DATA_Micro,n_steps_in,n_steps_out)
    firsttrain_dataset3=dataset_load(firsttrain3)
    firstvalid_dataset3=dataset_load(firstvalid3)
    train_dataset3=dataset_load(train_data3)
    test_dataset3=dataset_load(test_data3)
    firsttrain_loader3 = DataLoader(dataset=firsttrain_dataset3,batch_size=batch_size,shuffle=False)
    firstvalid_loader3 = DataLoader(dataset=firstvalid_dataset3,batch_size=batch_size,shuffle=False)
    train_loader3 = DataLoader(dataset=train_dataset3,batch_size=batch_size,shuffle=False)
    test_loader3 = DataLoader(dataset=test_dataset3,batch_size=10000,shuffle=False)
    best_hyperparameters3=Calibration3(True,'LSTM',firsttrain_loader3,firstvalid_loader3,n_features3)
    best_model3=Train_model3(True,'LSTM',best_hyperparameters3,train_loader3,n_features3)
    pred_BGru=scaler.inverse_transform([test_predict3(best_model3,test_loader3)]).ravel()
    
    #RegNetwork-MLP
    best_hyperparameters4=Calibration3(False,'LSTM',firsttrain_loader3,firstvalid_loader3,n_features3)
    best_model4=Train_model3(False,'LSTM',best_hyperparameters4,train_loader3,n_features3)
    pred_MLP=scaler.inverse_transform([test_predict3(best_model4,test_loader3)]).ravel()
    
    #SES
    SES_test=SES(DATA_Micro.Quantite[:],evaluate_SES(DATA_Micro,n1,n2))
    SES_pred=SES_test['Forecast'][n2:]
    
    #Croston
    Cr_test=Croston(DATA_Micro.Quantite[0:],alpha=evaluate_CR(DATA_Micro,n1,n2))
    Cr_pred=Cr_test['Forecast'][n2:-1]
    
    #SBA
    Opt_SBA=evaluate_SBA(DATA_Micro,n1,n2)
    SBA_test=SBA(DATA_Micro.Quantite[0:],alpha=Opt_SBA[0],beta=Opt_SBA[1])
    SBA_pred=SBA_test['Forecast'][n2:-1]
    
    #Naive
    naiv_pred=DATA_Micro['Quantite'][n2-Nlags:-Nlags].values
    
    return(prediction_HM,pred_MLP,pred_BGru,SES_pred,Cr_pred,SBA_pred,naiv_pred)

In [ ]:
path="./gen data/"
product="Prd1"
prediction_HM,pred_MLP,pred_BGru,SES_pred,Cr_pred,SBA_pred,naiv_pred=main(path,product)

# Saving results in csv file

In [ ]:
def PB_all(DATA_Micro,n2):
    L_PB=[]
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:],'phm':prediction_HM.ravel(),'pNN':pred_MLP.ravel(),'pRNN':pred_BGru.ravel(),'pses':SES_pred,'pcroston':Cr_pred,'pSBA':SBA_pred,'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_HM=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2))['phm']
    except:
        Pb_HM=0
    L_PB.append(Pb_HM)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pRNN':pred_BGru.ravel(),'pses':SES_pred,'pcroston':Cr_pred,'pSBA':SBA_pred,'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    try:
        Pb_NN=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pNN']
    except:
        Pb_NN=0
    L_PB.append(Pb_NN)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pses':SES_pred,'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pRNN':pred_BGru.ravel(),'pcroston':Cr_pred,'pSBA':SBA_pred,'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_SES=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pses']
    except:
        Pb_SES=0
    L_PB.append(Pb_SES)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pcroston':Cr_pred,'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pses':SES_pred,'pSBA':SBA_pred,'pRNN':pred_BGru.ravel(),'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_CR=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pcroston']
    except:
        Pb_CR=0
    L_PB.append(Pb_CR)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pSBA':SBA_pred,'pcroston':Cr_pred,'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pses':SES_pred,'pRNN':pred_BGru.ravel(),'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_SBA=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pSBA']
    except:
        Pb_SBA=0
    L_PB.append(Pb_SBA)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pRNN':pred_BGru.ravel(),'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pses':SES_pred,'pcroston':Cr_pred,'pSBA':SBA_pred,'pNaive':naiv_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_RNN=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pRNN']
    except:
        Pb_RNN=0
    L_PB.append(Pb_RNN)
    PB=pd.DataFrame({'pt':DATA_Micro['Quantite'][n2:-1],'pNaive':naiv_pred,'pRNN':pred_BGru.ravel(),'pNN':pred_MLP.ravel(),'phm':prediction_HM.ravel(),'pses':SES_pred,'pcroston':Cr_pred,'pSBA':SBA_pred})
    PB["error"]=PB.apply(lambda row: pb_func(abs(row[1:]-row[0])),axis=1)
    PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1)
    try:
        Pb_Naive=(PB['error'].value_counts()/(DATA_Micro.shape[0]-n2-1))['pNaive']
    except:
        Pb_Naive=0
    L_PB.append(Pb_Naive)
    return L_PB

In [ ]:
DATA_Micro=pd.read_csv(path+product+".csv", sep=';',encoding = "ISO-8859-1")
n1=np.where(DATA_Micro['PeriodsSepLastTwoNnZeroDemands']>0)[0][1]
n2=2*n1+int((DATA_Micro.shape[0]-n1)*90/100)
Train_X=DATA_Micro['Quantite'][n1:n2].values
Test_Y=DATA_Micro['Quantite'][n2:].values
ADI=(DATA_Micro['ZNZDemand'].value_counts()[1]+DATA_Micro['ZNZDemand'].value_counts()[0])/DATA_Micro['ZNZDemand'].value_counts()[1]
Test_df=(DATA_Micro[DATA_Micro['ZNZDemand']==1][['Quantite','Interval']]).dropna(axis=0)
CV2=(Test_df['Quantite'].std()/Test_df['Quantite'].mean())**2
n_obs=DATA_Micro.shape[0]-n1

In [ ]:
L_PB=PB_all(DATA_Micro,n2)
PB=False

In [ ]:
PB=False

In [ ]:
if PB:
    header=['product','Samples','ADI','CV2','MASE_HM','MASE_NN','MASE_SES','MASE_Cr','MASE_SBA','MASE_RNN','MASE_Naive',
        'RMSSE_HM','RMSSE_NN','RMSSE_SES','RMSSE_Cr','RMSSE_SBA','RMSSE_RNN','RMSSE_Naive',
        'PB_HM','PB_NN','PB_SES','PB_Cr','PB_SBA','PB_RNN','PB_Naive']
    dicti={
        'product':product,
        'Samples':n_obs,
        'ADI':ADI,
        'CV2':CV2,
        'MASE_HM':MASE(Train_X,Test_Y,prediction_HM),
        'MASE_NN':MASE(Train_X,Test_Y,pred_MLP),
        'MASE_SES':MASE(Train_X,Test_Y,SES_pred.values),
        'MASE_Cr':MASE(Train_X,Test_Y,Cr_pred),
        'MASE_SBA':MASE(Train_X,Test_Y,SBA_pred),
        'MASE_RNN':MASE(Train_X,Test_Y,pred_BGru),
        'MASE_Naive':MASE(Train_X,Test_Y,naiv_pred),
        'RMSSE_HM':RMSSE(Train_X,Test_Y,prediction_HM),
        'RMSSE_NN':RMSSE(Train_X,Test_Y,pred_MLP),
        'RMSSE_SES':RMSSE(Train_X,Test_Y,SES_pred),
        'RMSSE_Cr':RMSSE(Train_X,Test_Y,Cr_pred),
        'RMSSE_SBA':RMSSE(Train_X,Test_Y,SBA_pred),
        'RMSSE_RNN':RMSSE(Train_X,Test_Y,pred_BGru),
        'RMSSE_Naive':RMSSE(Train_X,Test_Y,naiv_pred),
        'PB_HM':L_PB[0],
        'PB_NN':L_PB[1],
        'PB_SES':L_PB[2],
        'PB_Cr':L_PB[3],
        'PB_SBA':L_PB[4],
        'PB_RNN':L_PB[5],
        'PB_Naive':L_PB[6]
        }
else:
    header=['product','Samples','ADI','CV2','MASE_HM','MASE_NN','MASE_SES','MASE_Cr','MASE_SBA','MASE_RNN','MASE_Naive',
        'RMSSE_HM','RMSSE_NN','RMSSE_SES','RMSSE_Cr','RMSSE_SBA','RMSSE_RNN','RMSSE_Naive']
    dicti={
        'product':product,
        'Samples':n_obs,
        'ADI':ADI,
        'CV2':CV2,
        'MASE_HM':MASE(Train_X,Test_Y,prediction_HM),
        'MASE_NN':MASE(Train_X,Test_Y,pred_MLP),
        'MASE_SES':MASE(Train_X,Test_Y,SES_pred.values),
        'MASE_Cr':MASE(Train_X,Test_Y,Cr_pred),
        'MASE_SBA':MASE(Train_X,Test_Y,SBA_pred),
        'MASE_RNN':MASE(Train_X,Test_Y,pred_BGru),
        'MASE_Naive':MASE(Train_X,Test_Y,naiv_pred),
        'RMSSE_HM':RMSSE(Train_X,Test_Y,prediction_HM),
        'RMSSE_NN':RMSSE(Train_X,Test_Y,pred_MLP),
        'RMSSE_SES':RMSSE(Train_X,Test_Y,SES_pred),
        'RMSSE_Cr':RMSSE(Train_X,Test_Y,Cr_pred),
        'RMSSE_SBA':RMSSE(Train_X,Test_Y,SBA_pred),
        'RMSSE_RNN':RMSSE(Train_X,Test_Y,pred_BGru),
        'RMSSE_Naive':RMSSE(Train_X,Test_Y,naiv_pred)
        }

In [ ]:
# saving results
try:
    df=pd.read_csv('Reg_res.csv')
    new=False
except:
    new=True
if new:
    with open('Reg_res.csv','w') as fd:
        writer = csv.writer(fd)
        writer.writerow(header)
        writer = csv.DictWriter(fd, fieldnames=header)
        writer.writerow(dicti)
else:
    with open('Reg_res.csv','a',newline='') as fd:
        writer = csv.DictWriter(fd, fieldnames=header)
        writer.writerow(dicti)